# 4. Chapter: AI Agents

We've explored the incredible capabilities of LLMs, the art of prompting, the integration of tools, and the generation of structured outputs. With this strong foundation, we're now ready to dive into something even more powerful - creating our very own LLM-powered AI agent. Here are just a few examples of what they can be used for: 
- **Customer support**: Provide instant, accurate, and empathetic responses to customer inquiries, improving satisfaction and efficiency. 
- **Content creation**: Generate high-quality articles, social media posts, and marketing copy with ease. 
- **Research assistance**: Summarize complex documents, find relevant studies, and provide insightful analysis. 
- **Personal assistants**: Manage schedules, set reminders, and help you stay organized. 
- **Creative expression**: Write stories, poems, and even act as a brainstorming partner for new ideas. 
- **Data analysis**: Process and interpret large datasets to extract meaningful patterns and insights.  

With the ability to configure your LLM-powered AI agent for different scenarios and outputs, you unlock an incredible tool that can adapt to various roles and industries. The potential applications are vast, limited only by your imagination and creativity.

In [1]:
from __future__ import annotations
import sys
import tiktoken
import requests
from enum import Enum
from typing import Any
from pydantic import BaseModel, Field, ValidationError, create_model
from datetime import datetime, timedelta
from loguru import logger
from language_models.agent.output_parser import (
    FINAL_ANSWER_INSTRUCTIONS,
    AgentOutputParser,
    LLMSingleCompletionFinalAnswer,
    LLMChainOfThoughtFinalAnswer,
    LLMToolUse,
    OutputType,
    get_schema_from_args,
)
from language_models.models.llm import ChatMessage, ChatMessageRole, OpenAILanguageModel
from language_models.tools.tool import Tool
from language_models.models.llm import ChatMessage
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
logger.remove()
logger.add(sys.stderr, format="{message}", level="INFO")

proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1000,
    temperature=0.2,
)

First, we define instructions for an AI agent to follow when responding to different types of prompts:
- **Single completion:** Provide a straightforward answer without using any tools. For example, if the prompt is "What's the capital of France?", respond with "The capital of France is Paris."  
- **Chain-of-Thought with tools:** Multi-step tasks that require tools. If the prompt is "Find the current weather in NYC and suggest an outfit," first, query a weather API to get the current weather (for instance, "22°C, partly cloudy"). Then suggest an appropriate outfit based on the weather, such as "Wear a t-shirt and a light jacket."  
- **Chain-of-Thought without tools:** Tasks that require (complex) structured outputs. If you want error correction logic due to the possibility of mistakes by the LLM.

In [4]:
SINGLE_COMPLETION_INSTRUCTIONS = """### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
<response to the prompt>
```"""

CHAIN_OF_THOUGHT_INSTRUCTIONS_WITH_TOOLS = """### Tools ###

You have access to the following tools:
{tools}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>

When you know the final answer to the user's query you should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```"""

CHAIN_OF_THOUGHT_INSTRUCTIONS_WITHOUT_TOOLS = """### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```"""

Next, we define a `Chat` class that manages the chat history and a list of intermediate steps. Given the model's inability to recall previous conversations, we introduce a mechanism to track and provide previous steps to the model. This ensures that the model can make informed decisions on what to do next or when to provide the user with the final answer.

In [5]:
class Chat(BaseModel):
    """Class that implements the chat history.

    Attributes:
        messages: The conversation history.
        steps: The intermediate steps.
    """

    messages: list[ChatMessage]
    steps: list[str] = []

    def update(self, prompt: str) -> None:
        """Modifies the user prompt to include intermediate steps."""
        self.messages[-1].content = "\n\n".join([prompt, f"This was your previous work:\n{'\n\n'.join(self.steps)}"])

    def reset(self) -> None:
        """Resets the chat."""
        self.messages = [self.messages[0]]
        self.steps = []

We count the number of tokens in a conversation history to ensure it stays within the predefined limits for different models.

In [6]:
MODEL_TOKEN_LIMIT = {
    "gpt-4": 8192,
    "gpt-4-32k": 32768,
    "gpt-35-turbo": 4096,
    "gpt-35-turbo-16k": 16385,
}

def num_tokens_from_messages(messages: list[ChatMessage]) -> int:
    """Counts the number of tokens in the conversation history."""
    encoding = tiktoken.get_encoding("cl100k_base")
    tokens_per_message = 3
    tokens_per_name = 1
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.model_dump().items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

Now, we create an `Agent` to encapsulate an LLM, providing methods for managing conversation context, parsing outputs, and executing the agent logic. 

It's important to recognize that LLMs can sometimes produce errors in their output, such as generating incorrect JSON or invalid tool input arguments. To address this, it is crucial to highlight these errors to the LLM in hopes that it can correct them. However, the LLM may not always be able to fix its own mistakes. In such instances, after several iterations of feedback and corrections, we may need to provide a fallback response.

This iterative process, combined with structured output formats and tool usage, forms the basis of an LLM-powered AI agent. While it may seem like magic in demonstrations, the underlying mechanism is actually quite straightforward.

![ReAct prompting](./assets/img/react.png)

In [7]:
class PromptingStrategy(str, Enum):
    SINGLE_COMPLETION = "single completion"
    CHAIN_OF_THOUGHT = "chain-of-thought"

class AgentOutput(BaseModel):
    """Class that represents the agent output."""

    prompt: str
    final_answer: (
        str
        | int
        | float
        | dict[str, Any]
        | BaseModel
        | list[str]
        | list[int]
        | list[float]
        | list[dict[str, Any]]
        | list[BaseModel]
        | None
    )

class Agent(BaseModel):
    """Class that implements an LLM-powered AI agent.

    Attributes:
        llm: The OpenAI LLM to use (in practice any model you choose).
        tools: The tools the LLM can use.
        prompt: The task prompt the LLM should solve.
        prompt_variables: The task prompt variables that are needed to run the agent.
        output_parser: The parser that handles LLM responses.
        chat: The chat history.
        prompt_strategy: The prompting strategy to use
            (single completion for input/output type of queries, chain-of-thought for multi-step queries that involve tools).
        iterations: The number of steps the LLM can take to solve the user query.
        verbose: Enable logging.
    """

    llm: OpenAILanguageModel
    tools: dict[str, Tool] | None
    prompt: str
    prompt_variables: list[str]
    output_parser: AgentOutputParser
    chat: Chat
    prompting_strategy: PromptingStrategy
    iterations: int = 10
    verbose: bool

    def trim_conversation(self) -> None:
        """Trims the chat messages to fit the LLM context length."""
        num_tokens = num_tokens_from_messages(self.chat.messages)
        while num_tokens + self.llm.max_tokens >= MODEL_TOKEN_LIMIT[self.llm.model]:
            del self.chat.messages[1]
            num_tokens = num_tokens_from_messages(self.chat.messages)

    def parse_output(self, output: str) -> LLMToolUse | LLMSingleCompletionFinalAnswer | LLMChainOfThoughtFinalAnswer:
        """Parses the LLM output."""
        try:
            output = self.output_parser.parse(output)
            observation = None
        except (ValueError, ValidationError) as error:
            output = None
            observation = error
        return output, observation

    def invoke(self, prompt: dict[str, Any]) -> AgentOutput:
        """Runs the agent given a prompt."""
        prompt = self.prompt.format(**{variable: prompt.get(variable) for variable in self.prompt_variables})
        self.chat.messages.append(ChatMessage(role=ChatMessageRole.USER, content=prompt))
        self.chat.steps = []

        iteration = 0
        while iteration <= self.iterations:
            self.trim_conversation()
            output = self.llm.get_completion(self.chat.messages)
            output, observation = self.parse_output(output)

            if self.prompting_strategy == PromptingStrategy.CHAIN_OF_THOUGHT:
                if output is not None:
                    if self.verbose:
                        logger.opt(colors=True).info(f"<b><fg #2D72D2>Thought</fg #2D72D2></b>: {output.thought}")

                    self.chat.steps.append(f"Thought: {output.thought}")

                    if isinstance(output, LLMChainOfThoughtFinalAnswer):
                        if self.verbose:
                            logger.opt(colors=True).success(f"<b><fg #32A467>Final Answer</fg #32A467></b>: {output.final_answer}")

                        self.chat.messages.append(
                            ChatMessage(role=ChatMessageRole.ASSISTANT, content=str(output.final_answer))
                        )
                        return AgentOutput(prompt=prompt, final_answer=output.final_answer)

                    else:
                        if self.tools is not None:
                            if self.verbose:
                                logger.opt(colors=True).info(f"<b><fg #EC9A3C>Tool</fg #EC9A3C></b>: {output.tool}")
                                logger.opt(colors=True).info(
                                    f"<b><fg #EC9A3C>Tool Input</fg #EC9A3C></b>: {output.tool_input}"
                                )

                            tool = self.tools.get(output.tool)
                            if tool is not None:
                                tool_output = tool.invoke(output.tool_input, self.verbose)
                                observation = f"Tool Output: {tool_output}"
                                if self.verbose:
                                    logger.opt(colors=True).info(f"<b><fg #EC9A3C>Tool Output</fg #EC9A3C></b>: {tool_output}")

                                self.chat.steps.append(f"Tool: {tool.name}")
                                self.chat.steps.append(f"Tool Input: {output.tool_input}")

                            else:
                                tool_names = ", ".join(list(self.tools.keys()))
                                observation = f"{output.tool} tool doesn't exist. Try one of these tools: {tool_names}"

                self.chat.steps.append(f"Observation: {observation}")
                self.chat.update(prompt)

            else:
                if isinstance(output, LLMSingleCompletionFinalAnswer):
                    if self.verbose:
                        logger.opt(colors=True).success(f"<b><fg #32A467>Final Answer</fg #32A467></b>: {output.final_answer}")

                    self.chat.messages.append(
                        ChatMessage(role=ChatMessageRole.ASSISTANT, content=str(output.final_answer))
                    )
                    return AgentOutput(prompt=prompt, final_answer=output.final_answer)

            iteration += 1

        if self.output_parser.output_type == OutputType.STRUCT:
            final_answer = {key: None for key in self.output_parser.output_schema.model_json_schema()["properties"]}
        elif self.output_parser.output_type == OutputType.ARRAY_STRUCT:
            final_answer = [{key: None for key in self.output_parser.output_schema.model_json_schema()["properties"]}]
        elif self.output_parser.output_type in (OutputType.OBJECT, OutputType.ARRAY_OBJECT):
            fields = self.output_parser.output_schema.__annotations__
            optional_fields = {field: (data_type | None, None) for field, data_type in fields.items()}
            model = create_model(self.output_parser.output_schema.__name__, **optional_fields)
            final_answer = model() if self.output_parser.output_type == OutputType.OBJECT else [model()]
        else:
            final_answer = None

        if self.verbose:
            logger.opt(colors=True).warning(f"<b><fg #CD4246>Final Answer</fg #CD4246></b>: {final_answer}")

        return AgentOutput(prompt=prompt, final_answer=final_answer)

    @classmethod
    def create(
        cls,
        llm: OpenAILanguageModel,
        system_prompt: str,
        prompt: str,
        prompt_variables: list[str],
        output_type: OutputType,
        output_schema: type[BaseModel] | str | None = None,
        tools: list[Tool] | None = None,
        prompting_strategy: PromptingStrategy = PromptingStrategy.CHAIN_OF_THOUGHT,
        verbose: bool = True,
    ) -> Agent:
        """Creates an instance of the Agent."""
        if prompting_strategy == PromptingStrategy.CHAIN_OF_THOUGHT:
            if tools is None:
                instructions = CHAIN_OF_THOUGHT_INSTRUCTIONS_WITHOUT_TOOLS
                tool_use = False
                tools = None
                iterations = 5
            else:
                instructions = CHAIN_OF_THOUGHT_INSTRUCTIONS_WITH_TOOLS.format(
                    tools="\n\n".join([str(tool) for tool in tools])
                )
                tool_use = True
                tools = {tool.name: tool for tool in tools}
                iterations = max(5, len(tools) * 2)
        else:
            instructions = SINGLE_COMPLETION_INSTRUCTIONS
            tool_use = False
            iterations = 1

        if output_type in (OutputType.OBJECT, OutputType.ARRAY_OBJECT):
            if output_schema is None:
                raise ValueError(f"When using {output_type} as the output type a schema must be provided.")

            args = output_schema.model_json_schema()["properties"]
            final_answer_instructions = FINAL_ANSWER_INSTRUCTIONS[output_type].format(
                output_schema=get_schema_from_args(args)
            )
        elif output_type in (OutputType.DATE, OutputType.TIMESTAMP):
            final_answer_instructions = FINAL_ANSWER_INSTRUCTIONS[output_type].format(output_schema=output_schema)
        else:
            final_answer_instructions = FINAL_ANSWER_INSTRUCTIONS[output_type]

        chat = Chat(
            messages=[
                ChatMessage(
                    role=ChatMessageRole.SYSTEM,
                    content="\n\n".join([system_prompt, instructions, final_answer_instructions]),
                )
            ]
        )

        output_parser = AgentOutputParser(
            output_type=output_type,
            output_schema=output_schema,
            prompting_strategy=prompting_strategy,
            tool_use=tool_use,
        )

        return Agent(
            llm=llm,
            tools=tools,
            prompt=prompt,
            prompt_variables=prompt_variables,
            output_parser=output_parser,
            chat=chat,
            prompting_strategy=prompting_strategy,
            iterations=iterations,
            verbose=verbose,
        )

## Single Completion

The following code creates an AI agent that can answer user questions based on the internal knowledge of the LLM. It's designed to provide precise and detailed responses by leveraging a single completion strategy.

In [8]:
system_prompt = "You are an AI assistant designed to help users with a variety of tasks."

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

In [9]:
output = agent.invoke({"question": "Hi, can you help me understand the basics of quantum computing?"})

Final Answer: Sure, I'd be happy to help you understand the basics of quantum computing!

Quantum computing is a type of computation that uses quantum bits, or qubits, to process information. Unlike classical bits, which can be either a 0 or a 1, a qubit can be both a 0 and a 1 at the same time, thanks to a property called superposition.

Another key principle of quantum computing is entanglement, which allows qubits that are entangled to be linked together in such a way that the state of one qubit can depend on the state of another, no matter how far they are separated.

These two properties give quantum computers the potential to process a much higher amount of data compared to classical computers. Quantum computers can potentially solve certain types of problems much more efficiently than classical computers, such as factoring large numbers or simulating the behavior of quantum particles.

However, building a practical quantum computer is a huge technological challenge due to issues

In [10]:
print(output.final_answer)

Sure, I'd be happy to help you understand the basics of quantum computing!

Quantum computing is a type of computation that uses quantum bits, or qubits, to process information. Unlike classical bits, which can be either a 0 or a 1, a qubit can be both a 0 and a 1 at the same time, thanks to a property called superposition.

Another key principle of quantum computing is entanglement, which allows qubits that are entangled to be linked together in such a way that the state of one qubit can depend on the state of another, no matter how far they are separated.

These two properties give quantum computers the potential to process a much higher amount of data compared to classical computers. Quantum computers can potentially solve certain types of problems much more efficiently than classical computers, such as factoring large numbers or simulating the behavior of quantum particles.

However, building a practical quantum computer is a huge technological challenge due to issues like maintain

In [11]:
output = agent.invoke({"question": "Alex has three times as many apples as Ben. Together, they have 24 apples. How many apples does each person have?"})

Final Answer: Let's denote the number of apples Ben has as x. According to the problem, Alex has 3 times as many apples as Ben, so Alex has 3x apples. Together, they have 24 apples. So we can set up the following equation:

x (Ben's apples) + 3x (Alex's apples) = 24

Solving this equation gives:

4x = 24
x = 24 / 4
x = 6

So, Ben has 6 apples and Alex, having three times as many, has 18 apples.


In [12]:
print(output.final_answer)

Let's denote the number of apples Ben has as x. According to the problem, Alex has 3 times as many apples as Ben, so Alex has 3x apples. Together, they have 24 apples. So we can set up the following equation:

x (Ben's apples) + 3x (Alex's apples) = 24

Solving this equation gives:

4x = 24
x = 24 / 4
x = 6

So, Ben has 6 apples and Alex, having three times as many, has 18 apples.


## Chain-of-Thought

The following code configures an LLM to specialize in answering earthquake-related questions by simulating the expertise of a United States Geological Survey (USGS) expert. It integrates various tools, including those for earthquake information and current date retrieval, to enhance its responses.

In [13]:
def current_date() -> datetime:
    return datetime.now()

current_date_tool = Tool(
    function=current_date,
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

In [14]:
class USGeopoliticalSurveyEarthquakeAPI(BaseModel):
    """Class that implements the API interface."""

    start_time: str = Field(
        None,
        description=(
            "Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    end_time: str = Field(
        None,
        description=(
            "Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    limit: int = Field(
        20000,
        description=(
            "Limit the results to the specified number of events. NOTE: The service limits queries to 20000,"
            + " and any that exceed this limit will generate a HTTP response code 400 Bad Request."
        ),
    )
    min_depth: int = Field(
        -100,
        description="Limit to events with depth more than the specified minimum.",
    )
    max_depth: int = Field(
        1000,
        description="Limit to events with depth less than the specified maximum.",
    )
    min_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude larger than the specified minimum.",
    )
    max_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude smaller than the specified maximum.",
    )
    alert_level: str = Field(
        None,
        description=(
            "Limit to events with a specific PAGER alert level."
            + " The allowed values are: alert_level=green Limit to events with PAGER"
            + ' alert level "green". alert_level=yellow Limit to events with PAGER alert level "yellow".'
            + ' alert_level=orange Limit to events with PAGER alert level "orange".'
            + ' alert_level=red Limit to events with PAGER alert level "red".'
        ),
    )

def get_earthquakes(
    endpoint: str,
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    params = {
        "format": "geojson",
        "starttime": start_time,
        "endtime": end_time,
        "limit": limit,
        "mindepth": min_depth,
        "maxdepth": max_depth,
        "minmagnitude": min_magnitude,
        "maxmagnitude": max_magnitude,
        "alertlevel": alert_level,
        "eventtype": "earthquake",
    }
    response = requests.get(
        f"https://earthquake.usgs.gov/fdsnws/event/1/{endpoint}",
        params=params,
        timeout=None,
    )
    return response.json()

def query_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="query",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

def count_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="count",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

In [15]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date_tool, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
)

The following question demonstrates how the LLM processes a user's query about the number of earthquakes that occurred on the current day. Using the provided tools, the LLM first retrieves the current date and time. This initial step is crucial for identifying and counting the seismic activities for the day. This process shows the power of [Chain-of-Thought](https://arxiv.org/abs/2201.11903), particularly [ReAct](https://arxiv.org/abs/2210.03629), which enables the LLM to tackle multi-step problems. By breaking down the task into smaller, manageable steps - first obtaining the current date, then querying the relevant seismic data - the LLM can deliver correct answers.

In [16]:
output = agent.invoke({"question": "How many earthquakes occurred today?"})

Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start time as the beginning of today and the end time as the current time.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-10 14:01:07.056624
Thought: Now that I have the current date and time, I can use the "Count Earthquakes" tool to count the number of earthquakes that occurred today. The start time will be the beginning of today (2024-08-10 00:00:00) and the end time will be the current time (2024-08-10 14:01:07).
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-10T00:00:00', 'end_time': '2024-08-10T14:01:07'}
Tool Output: {'count': 106, 'maxAllowed': 20000}
Thought: The output from the "Count Earthquakes" tool shows that there have been 106 earthquakes today.
Final Answer: There have been 106 earthquakes today.


In [17]:
print(output.final_answer)

There have been 106 earthquakes today.


When we provide the LLM with a follow-up question where ```Show me 3``` refers to the earthquakes that occurred today, it uses the context and continuity from the chat history to understand that "today" pertains to the current day. This capability allows the LLM to skip re-fetching the current date and directly query the earthquakes that happened on the same day, based on the date from the previous interaction.

In [18]:
output = agent.invoke({"question": "Show me 3."})

Thought: To show details of 3 earthquakes that occurred today, I can use the "Query Earthquakes" tool. I will use the same start and end times as before, and set the limit to 3.
Tool: Query Earthquakes
Tool Input: {'start_time': '2024-08-10T00:00:00', 'end_time': '2024-08-10T14:01:07', 'limit': 3}
Tool Output: {'type': 'FeatureCollection', 'metadata': {'generated': 1723291276000, 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2024-08-10T00%3A00%3A00&endtime=2024-08-10T14%3A01%3A07&limit=3&mindepth=-100&maxdepth=1000&eventtype=earthquake', 'title': 'USGS Earthquakes', 'status': 200, 'api': '1.14.1', 'limit': 3, 'offset': 1, 'count': 3}, 'features': [{'type': 'Feature', 'properties': {'mag': 1.6, 'place': '61 km E of Port Alsworth, Alaska', 'time': 1723290720427, 'updated': 1723290845646, 'tz': None, 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak024a8zpb1m', 'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak024a8zpb1m&f

In [19]:
print(output.final_answer)

Here are the details of 3 earthquakes that occurred today:

1. A magnitude 1.6 earthquake occurred 61 km east of Port Alsworth, Alaska. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/ak024a8zpb1m).

2. A magnitude 0.95 earthquake occurred 11 km southwest of Salton City, California. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/ci40696223).

3. A magnitude -0.1 earthquake occurred 30 km east-northeast of Beatty, Nevada. More details can be found [here](https://earthquake.usgs.gov/earthquakes/eventpage/nn00882187).


The LLM can also handle straightforward informational questions that don't require the use of additional tools. In this case, the question about the possibility of MegaQuakes (magnitude 10 or larger) can be answered directly by the model without invoking any tools. This shows the LLM's ability to recognize when it can rely on its internal knowledge base to provide an accurate response.

In [20]:
output = agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

Thought: The user is asking about the possibility of a magnitude 10 or larger earthquake, often referred to as a "MegaQuake". This is a question about the theoretical limits of earthquake magnitude, which is determined by the size of the fault on which it occurs.
Final Answer: Theoretically, "MegaQuakes" of magnitude 10 or larger could occur. However, they are extremely unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs. Therefore, a magnitude 10 earthquake would require a fault that is several thousand miles long. There are no known faults of this length on Earth. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. It's important to note that the magnitude scale is logarithmic, which means a magnitude 10 earthquake would release 31.6 times more energy than a magnitude 9 earthquake, and 1,000 times more energy than a magnitude 8 earthquake.


In [21]:
print(output.final_answer)

Theoretically, "MegaQuakes" of magnitude 10 or larger could occur. However, they are extremely unlikely. The magnitude of an earthquake is related to the length of the fault on which it occurs. Therefore, a magnitude 10 earthquake would require a fault that is several thousand miles long. There are no known faults of this length on Earth. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. It's important to note that the magnitude scale is logarithmic, which means a magnitude 10 earthquake would release 31.6 times more energy than a magnitude 9 earthquake, and 1,000 times more energy than a magnitude 8 earthquake.
